# 런타임에 체인 내부 구성하기

이번 튜토리얼은 Chain 을 호출시 다양한 옵션을 동적으로 설정할 수 있는 방법을 알아보겠습니다.

다음의 2가지 방식으로 동적 구성을 할 수 있습니다.

- 첫째, `configurable_fields` 메서드입니다. 이 메서드를 통해 실행 가능한 객체의 특정 필드를 구성할 수 있습니다.

- 둘째, `configurable_alternatives` 메서드입니다. 이 메서드를 사용하면 런타임 중에 설정할 수 있는 특정 실행 가능한 객체에 대한 대안을 나열할 수 있습니다.


## configurable_fields

`configurable_fields` 는 시스템의 설정 값을 정의하는 필드를 의미합니다.

## 동적 속성 지정

`ChatOpenAI` 을 사용할 때, 우리는 `model_name` 와 같은 설정을 조정할 수 있습니다.

`model_name` 은 GPT 의 버전을 명시할 때 사용하는 속성입니다. 예를 들어, `gpt-4o`, `gpt-4o-mini` 등을 설정하여 모델을 선택할 수 있습니다.

만약, 고정된 `model_name` 이 아닌 동적으로 모델을 지정하고 싶을 때는 다음과 같이 `ConfigurableField` 를 활용하여 동적으로 설정할 수 있는 속성 값으로 변환할 수 있습니다.


In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH13-LCEL-Advanced")

`configurable_fields` 메서드를 사용하여 `model_name` 속성을 동적 구성 가능한 필드로 지정합니다.


In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, model_name="gpt-4o")

model.invoke("대한민국의 수도는 어디야?").__dict__

In [ ]:
model = ChatOpenAI(temperature=0).configurable_fields(
    model_name=ConfigurableField(  # model_name 은 원래 ChatOpenAI 의 필드입니다.
        id="gpt_version",  # model_name 의 id 를 설정합니다.
        name="Version of GPT",  # model_name 의 이름을 설정합니다.
        # model_name 의 설명을 설정합니다.
        description="Official model name of GPTs. ex) gpt-4o, gpt-4o-mini",
    )
)

`model.invoke()` 호출시 `config={"configurable": {"키": "값"}}` 형식으로 동적 지정할 수 있습니다.


In [ ]:
model.invoke(
    "대한민국의 수도는 어디야?",
    # gpt_version 을 gpt-3.5-turbo 로 설정합니다.
    config={"configurable": {"gpt_version": "gpt-3.5-turbo"}},
).__dict__

이번에는 `gpt-4o-mini` 모델을 사용해보겠습니다. 출력에 바뀐 모델을 확인하세요.


In [ ]:
model.invoke(
    # gpt_version 을 gpt-4o-mini 로 설정합니다.
    "대한민국의 수도는 어디야?",
    config={"configurable": {"gpt_version": "gpt-4o-mini"}},
).__dict__

`model` 객체의 `with_config()` 메서드를 사용하여 `configurable` 매개변수를 설정할 수도 있습니다. 이전과 동작하는 방식은 동일합니다.


In [ ]:
model.with_config(configurable={"gpt_version": "gpt-4o-mini"}).invoke(
    "대한민국의 수도는 어디야?"
).__dict__

또한 이 함수를 체인의 일부로 사용할 때에도 동일한 방식으로 활용할 수 있습니다.


In [ ]:
# 템플릿에서 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template("{x} 보다 큰 위의 난수를 선택합니다.")
chain = (
    prompt | model
)  # 프롬프트와 모델을 연결하여 체인을 생성합니다. 프롬프트의 출력이 모델의 입력으로 전달됩니다.

In [ ]:
chain.invoke({"x": 0}).__dict__  # 체인을 호출하고 입력 변수 "x"에 0을 전달합니다.

In [ ]:
# 체인을 호출할 때 설정을 지정하여 체인을 호출할 수 있습니다.
chain.with_config(configurable={"gpt_version": "gpt-4o"}).invoke({"x": 0}).__dict__

## HubRunnable: LangChain Hub의 설정 변경

`HubRunnable` 을 사용하면 Hub 에 등록된 프롬프트의 전환을 용이하게 합니다.


In [ ]:
from langchain.runnables.hub import HubRunnable

prompt = HubRunnable("teddynote/rag-prompt-korean").configurable_fields(
    # 소유자 저장소 커밋을 설정하는 ConfigurableField
    owner_repo_commit=ConfigurableField(
        # 필드의 ID
        id="hub_commit",
        # 필드의 이름
        name="Hub Commit",
        # 필드에 대한 설명
        description="Korean RAG prompt by teddynote",
    )
)
prompt

별도의 `with_config` 지정 없이 `prompt.invoke()` 메서드를 호출하면 처음 설정한 `"rlm/rag-prompt"` hub 에 등록된 프롬프트를 pull 하여 가져옵니다.


In [ ]:
# prompt 객체의 invoke 메서드를 호출하여 "question"과 "context" 매개변수를 전달합니다.
prompt.invoke({"question": "Hello", "context": "World"}).messages

In [ ]:
prompt.with_config(
    # hub_commit 을 teddynote/simple-summary-korean 으로 설정합니다.
    configurable={"hub_commit": "teddynote/simple-summary-korean"}
).invoke({"context": "Hello"})

## Configurable Alternatives: Runnable 객체 자체의 대안 설정

런타임에 설정할 수 있는 Runnable 에 대한 대안을 구성합니다.

**구성 가능한 대안들**

`ChatAnthropic` 의 구성 가능한 언어 모델은 다양한 작업과 컨텍스트에 적용할 수 있는 유연성을 제공합니다.

동적으로 설정(Config) 값을 변경하기 위하여 모델에 설정하는 파라미터를 ConfigurableField 객체로 설정합니다.

- `model`: 사용할 기본 언어 모델을 지정합니다.

- `temperature`: 0에서 1 사이의 값으로, 샘플링의 무작위성을 제어합니다. 값이 낮을수록 더 결정적이고 반복적인 출력이 생성되며, 값이 높을수록 더 다양하고 창의적인 출력이 생성됩니다.

### LLM 객체의 대안(alternatives) 설정 방법

LLM(Large Language Model)을 활용하여 이를 수행하는 방법을 살펴보겠습니다.

[참고]

- `ChatAnthropic` 모델을 사용하기 위하여 API KEY를 발급받아 설정해야합니다.
- 링크: https://console.anthropic.com/dashboard
- 아래 주석을 해제하고 API KEY를 설정하거나, `.env` 파일에 설정합니다.
  
`ANTHROPIC_API_KEY` 환경변수를 설정합니다.

In [ ]:
# import os

# os.environ["ANTHROPIC_API_KEY"] = "ANTHROPIC API KEY를 입력합니다."

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

llm = ChatAnthropic(
    temperature=0, model="claude-3-5-sonnet-20240620"
).configurable_alternatives(
    # 이 필드에 id를 부여합니다.
    # 최종 실행 가능한 객체를 구성할 때, 이 id를 사용하여 이 필드를 구성할 수 있습니다.
    ConfigurableField(id="llm"),
    # 기본 키를 설정합니다.
    # 이 키를 지정하면 위에서 초기화된 기본 LLM(ChatAnthropic)이 사용됩니다.
    default_key="anthropic",
    # 'openai'라는 이름의 새 옵션을 추가하며, 이는 `ChatOpenAI()`와 동일합니다.
    openai=ChatOpenAI(model="gpt-4o-mini"),
    # 'gpt4'라는 이름의 새 옵션을 추가하며, 이는 `ChatOpenAI(model="gpt-4")`와 동일합니다.
    gpt4o=ChatOpenAI(model="gpt-4o"),
    # 여기에 더 많은 구성 옵션을 추가할 수 있습니다.
)
prompt = PromptTemplate.from_template("{topic} 에 대해 간단히 설명해주세요.")
chain = prompt | llm

`chain.invoke()` 메서드를 기본 LLM 인 `ChatAnthropic` 을 활용한 체인을 호출합니다.


In [ ]:
# Anthropic을 기본으로 호출합니다.
chain.invoke({"topic": "뉴진스"}).__dict__

`chain.with_config(configurable={"llm": "모델"})`를 사용하여 사용할 `llm`으로 다른 모델을 지정할 수 있습니다.


In [ ]:
# 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"llm": "openai"}).invoke({"topic": "뉴진스"}).__dict__

체인의 설정을 변경하여 사용할 언어 모델을 `gpt4o` 로 지정합니다.


In [ ]:
# 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"llm": "gpt4o"}).invoke({"topic": "뉴진스"}).__dict__

체인의 설정을 변경하여 사용할 언어 모델을 `anthropic` 로 지정합니다.


In [ ]:
# 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"llm": "anthropic"}).invoke(
    {"topic": "뉴진스"}
).__dict__

## 프롬프트의 대안 설정 방법

프롬프트도 이전의 LLM 대안 설정 방법과 유사한 작업을 수행할 수 있습니다.


In [ ]:
# 언어 모델을 초기화하고 temperature를 0으로 설정합니다.
llm = ChatOpenAI(temperature=0)

prompt = PromptTemplate.from_template(
    "{country} 의 수도는 어디야?"  # 기본 프롬프트 템플릿
).configurable_alternatives(
    # 이 필드에 id를 부여합니다.
    ConfigurableField(id="prompt"),
    # 기본 키를 설정합니다.
    default_key="capital",
    # 'area'이라는 새로운 옵션을 추가합니다.
    area=PromptTemplate.from_template("{country} 의 면적은 얼마야?"),
    # 'population'이라는 새로운 옵션을 추가합니다.
    population=PromptTemplate.from_template("{country} 의 인구는 얼마야?"),
    # 'eng'이라는 새로운 옵션을 추가합니다.
    eng=PromptTemplate.from_template("{input} 을 영어로 번역해주세요."),
    # 여기에 더 많은 구성 옵션을 추가할 수 있습니다.
)

# 프롬프트와 언어 모델을 연결하여 체인을 생성합니다.
chain = prompt | llm

아무런 설정 변경이 없다면 기본 프롬프트가 입력됩니다.


In [ ]:
# config 변경 없이 체인을 호출합니다.
chain.invoke({"country": "대한민국"})

`with_config` 로 다른 프롬프트를 호출합니다.


In [ ]:
# with_config 로 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"prompt": "area"}).invoke({"country": "대한민국"})

In [ ]:
# with_config 로 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"prompt": "population"}).invoke({"country": "대한민국"})

이번에는 `eng` 프롬프트를 사용하여 번역을 요청합니다. 이때 전달할 입력 변수는 `input`입니다.


In [ ]:
# with_config 로 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"prompt": "eng"}).invoke({"input": "사과는 맛있어!"})

## 프롬프트 & LLM 모두 변경

프롬프트와 LLM을 사용하여 여러 가지 사항을 구성할 수 있습니다.

다음은 프롬프트와 LLM을 모두 사용하여 이를 수행하는 예시입니다.


In [ ]:
llm = ChatAnthropic(
    temperature=0, model="claude-3-5-sonnet-20240620"
).configurable_alternatives(
    # 이 필드에 id를 부여합니다.
    # 최종 실행 가능한 객체를 구성할 때, 이 id를 사용하여 이 필드를 구성할 수 있습니다.
    ConfigurableField(id="llm"),
    # 기본 키를 설정합니다.
    # 이 키를 지정하면 위에서 초기화된 기본 LLM(ChatAnthropic)이 사용됩니다.
    default_key="anthropic",
    # 'openai'라는 이름의 새 옵션을 추가하며, 이는 `ChatOpenAI(model="gpt-4o-mini")`와 동일합니다.
    openai=ChatOpenAI(model="gpt-4o-mini"),
    # 'gpt4'라는 이름의 새 옵션을 추가하며, 이는 `ChatOpenAI(model="gpt-4o")`와 동일합니다.
    gpt4=ChatOpenAI(model="gpt-4o"),
    # 여기에 더 많은 구성 옵션을 추가할 수 있습니다.
)

prompt = PromptTemplate.from_template(
    "{company} 에 대해서 20자 이내로 설명해 줘."  # 기본 프롬프트 템플릿
).configurable_alternatives(
    # 이 필드에 id를 부여합니다.
    ConfigurableField(id="prompt"),
    # 기본 키를 설정합니다.
    default_key="description",
    # 'founder'이라는 새로운 옵션을 추가합니다.
    founder=PromptTemplate.from_template("{company} 의 창립자는 누구인가요?"),
    # 'competitor'이라는 새로운 옵션을 추가합니다.
    competitor=PromptTemplate.from_template("{company} 의 경쟁사는 누구인가요?"),
    # 여기에 더 많은 구성 옵션을 추가할 수 있습니다.
)
chain = prompt | llm

In [ ]:
# with_config 로 설정 값을 지정하여 구성할 수 있습니다.
chain.with_config(configurable={"prompt": "founder", "llm": "openai"}).invoke(
    # 사용자가 제공한 회사에 대한 처리를 요청합니다.
    {"company": "애플"}
).__dict__

In [ ]:
# 하나만 구성하려는 경우
chain.with_config(configurable={"llm": "anthropic"}).invoke(
    {"company": "애플"}
).__dict__

In [ ]:
# 하나만 구성하려는 경우
chain.with_config(configurable={"prompt": "competitor"}).invoke(
    {"company": "애플"}
).__dict__

In [ ]:
# 하나만 구성하려는 경우
chain.invoke({"company": "애플"}).__dict__

## 설정 저장

구성된 체인을 별도의 객체로 쉽게 저장할 수 있습니다. 예를 들어, 특정 작업을 위해 사용자 정의된 체인을 구성한 후, 이를 재사용 가능한 객체로 저장함으로써 향후 유사한 작업에서 손쉽게 활용할 수 있습니다.


In [ ]:
# with_config 로 설정을 변경하여 생성한 체인을 별도의 변수에 저장합니다.
gpt4_competitor_chain = chain.with_config(
    configurable={"llm": "gpt4", "prompt": "competitor"}
)

In [ ]:
# 체인을 호출합니다.
gpt4_competitor_chain.invoke({"company": "애플"}).__dict__